# Notebook 2: Applying cuts with CutManager

In this notebook there is a simple example of how Galapago applies several cuts to be evaluated when reading an event.

This is usually done inside ```include/processHandler.py```. However this example is simplified with an event loop.

In [1]:
import os

if 'notebooks' in os.getcwd():
    os.chdir('..')

## 2.0 - CutManager and cut syntax

We first import everything that is needed:

In [2]:
import ROOT as r
import include.CutManager as CutManager
import include.Sample as Sample
import include.helper as helper

Welcome to JupyROOT 6.20/06


And we create an instance of the ```CutManager``` class:

In [3]:
cutmanager = CutManager.CutManager()

Cuts in CutManager are tuned to be used with Python. Each cut is basically a string variable that contains the expression that it is going to be evaluated. For evaluating each cut, we must use the python ```eval()``` function.

Cuts are usually defined as atributes of the ```CutManager``` class. We take as an example the mass cut applied on MM and EE:

In [4]:
print('Mass cut in MM:', cutmanager.MM_mass15)
print('Mass cut in EE:', cutmanager.EE_mass15)

Mass cut in MM: (ev.DMDM_mass[imm] > 15)
Mass cut in EE: (ev.EE_mass[iee] > 15)


### The syntax

For the cuts to work, there are two rules that must hold:

1. Every variable in the Ntuples must be accompanied with a prefix referring to the event that is evaluated. This prefix is set by default to ```'.ev'``` in ```processHandler.py```.


2. When the cut is applied on LL candidates, which are arrays, (i.e. EE_* or DMDM_* variables) the index must be specified. ```imm``` holds for the index of the MM candidate and ```iee``` for the index of the EE candidate.


Both ```ev```, ```imm``` and ```iee``` must be declared as python variables first. 

## 2.1 - Evaluating cuts

In order to evaluate the cuts, we load the same signal sample that was used in Notebooks 0 and 1 by reading the datacard with the ```include/Sample.py``` module:

In [5]:
# Access the Tree of one sample and print its information

treeSI = Sample.Tree( fileName = helper.selectSamples('dat/Samples_cern_fillingv2.dat', ['HXX_1000_150_100'], 'SI'), name = 'SI', isdata = 0 )
treeSI.printTree()

tree_block = treeSI.blocks[0]
tree_sample = tree_block.samples[0]
ttree = tree_sample.ttrees[0]

print("Tree entries: ", ttree.GetEntries())

selectSamples for  SI : List Of Samples: ['HXX_1000_150_100']
---> Found a match for HXX_1000_150_100 : HXX_1000_150_100mm   HXX(1000,150,100) /eos/user/f/fernance/LLP_Analysis/NTuples/2016_v3/HXX_1000_150_100mm/ 0.7739 , matchesName_= False , matchesRegExp True
######
Tree Name:  SI
Tree IsData:  0
######
This Tree contains the following Blocks
####################
Block Name:  HXX_1000_150_100mm
Block Color:  632
Block IsData:  0
####################
This block contains the following Samples
#################################
Sample Name:  HXX_1000_150_100mm
Sample Location:  /eos/user/f/fernance/LLP_Analysis/NTuples/2016_v3/HXX_1000_150_100mm/
Sample XSection:  0.7739
Sample IsData:  0
Sample LumWeight:  4.060335781741868e-06
#################################
Tree entries:  190600


We loop over 10 events looking for events that have at least 1 EE or MM candidate, and we evaluate the mass cut printed before in these. 

<em> Each cut is evaluated as a string with ```eval()``` function </em>

In [16]:
for n,ev in enumerate(ttree): # Looping over ev objects

    if n > 10: break
    
    print('-> Event:', str(n))
    
    if eval(cutmanager.haveEE):
        iee = 0 # EE index definition !!!
        print('   Number of EEs:', str(ev.nEE))
        print('   Mass of the first EE:', str(ev.EE_mass[iee]))
        print('     Is greater than 15? ', eval(cutmanager.EE_mass15))
    else:
        print('   Number of EEs:', str(ev.nEE), 'skipping...')
        
    if eval(cutmanager.haveMM):
        imm = 0 # MM index definition !!!
        print('   Number of MMs:', str(ev.nDMDM))
        print('   Mass of the first MM:', str(ev.DMDM_mass[iee]))
        print('     Is greater than 15? ', eval(cutmanager.MM_mass15))
    else:
        print('   Number of MMs:', str(ev.nDMDM), 'skipping...')

-> Event: 0
   Number of EEs: 1
   Mass of the first EE: 144.19895935058594
     Is greater than 15?  True
   Number of MMs: 1
   Mass of the first MM: 136.97764587402344
     Is greater than 15?  True
-> Event: 1
   Number of EEs: 0 skipping...
   Number of MMs: 0 skipping...
-> Event: 2
   Number of EEs: 1
   Mass of the first EE: 151.62594604492188
     Is greater than 15?  True
   Number of MMs: 0 skipping...
-> Event: 3
   Number of EEs: 0 skipping...
   Number of MMs: 1
   Mass of the first MM: 158.80325317382812
     Is greater than 15?  True
-> Event: 4
   Number of EEs: 0 skipping...
   Number of MMs: 0 skipping...
-> Event: 5
   Number of EEs: 0 skipping...
   Number of MMs: 2
   Mass of the first MM: 150.95260620117188
     Is greater than 15?  True
-> Event: 6
   Number of EEs: 0 skipping...
   Number of MMs: 1
   Mass of the first MM: 159.346923828125
     Is greater than 15?  True
-> Event: 7
   Number of EEs: 0 skipping...
   Number of MMs: 1
   Mass of the first MM: 138